In [1]:
import collections
import math
import os
import sys
import argparse
import random
from tempfile import gettempdir
import zipfile

import numpy as np
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf
import numpy as np
from tensorflow.contrib.tensorboard.plugins import projector


c:\users\leo\appdata\local\conda\conda\envs\gpu-tf\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
filename = "data/news.txt"

vocabulary = tf.compat.as_str(open(filename, encoding='utf8').read()).split()
vocab = list(set(vocabulary))


# Step 2: Build the dictionary and replace rare words with UNK token.
vocabulary_size = len(vocab)
print('Data size', len(vocabulary), ' Vocab Size', vocabulary_size)


def build_dataset(words, n_words):
    """Process raw inputs into a dataset."""
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(n_words - 1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        index = dictionary.get(word, 0)
        if index == 0:  # dictionary['UNK']
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary



Data size 4089439  Vocab Size 297836


In [3]:
data, count, dictionary, reverse_dictionary = build_dataset(
    vocabulary, vocabulary_size)
del vocabulary  # Hint to reduce memory.
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])

data_index = 0



Most common words (+UNK) [['UNK', 1], ('*', 239982), ('ነው', 56425), ('#', 45253), ('ላይ', 37050)]
Sample data [2752, 3645, 997, 50365, 1633, 44513, 1968, 417, 953, 113] ['ጋዜጠኛ', 'ተመስገን', 'ደሳለኝ', 'በጠበቃው', 'በአቶ', 'አምሐ', 'መኮንን', 'አማካይነት', 'በፌዴራል', 'ጠቅላይ']


In [4]:

# Step 3: Function to generate a training batch for the skip-gram model.
def generate_batch(batch_size, num_skips, skip_window):
#     batch_size = batch_size // 2
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1  # [ skip_window target skip_window ]
    buffer = collections.deque(
        maxlen=span)  # pylint: disable=redefined-builtin
    if data_index + span > len(data):
        data_index = 0
    buffer.extend(data[data_index:data_index + span])
    data_index += span
    for i in range(batch_size // num_skips):
        context_words = [w for w in range(span) if w != skip_window]
        words_to_use = random.sample(context_words, num_skips)
        for j, context_word in enumerate(words_to_use):
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j, 0] = buffer[context_word]
        if data_index == len(data):
            buffer.extend(data[0:span])
            data_index = span
        else:
            buffer.append(data[data_index])
            data_index += 1
            
#     sim_batch = np.zeros_like(batch)
#     sim_labels = np.zeros_like(labels)
#     for b_i in range(len(batch)):
#         b = batch[b_i]
#         sim = sim_data[b]
#         sim_context = np.random.choice(sim, 1)
#         for sim_word in sim_context:
#             sim_batch[b_i] = b
#             sim_labels[b_i, 0] = sim_word
    
#     for sim_i in range(len(sim_batch)):
#         rev_word = reverse_dictionary[sim_batch[sim_i]]
#         rev_label = reverse_dictionary[sim_labels[sim_i, 0]]
#     batch = np.hstack((batch, sim_batch))
#     labels = np.vstack((labels, sim_labels))
        
    # Backtrack a little bit to avoid skipping words in the end of a batch
    data_index = (data_index + len(data) - span) % len(data)
    return batch, labels


batch, labels = generate_batch(batch_size=8, num_skips=2, skip_window=2)
for i in range(len(batch)):
    print(batch[i], reverse_dictionary[batch[i]], '->', labels[i, 0],
          reverse_dictionary[labels[i, 0]])

# Step 4: Build and train a skip-gram model.


997 ደሳለኝ -> 2752 ጋዜጠኛ
997 ደሳለኝ -> 3645 ተመስገን
50365 በጠበቃው -> 3645 ተመስገን
50365 በጠበቃው -> 44513 አምሐ
1633 በአቶ -> 50365 በጠበቃው
1633 በአቶ -> 44513 አምሐ
44513 አምሐ -> 1968 መኮንን
44513 አምሐ -> 50365 በጠበቃው


In [38]:

batch_size = 128
embedding_size = 128  # Dimension of the embedding vector.
skip_window = 3  # How many words to consider left and right.
num_skips = 2  # How many times to reuse an input to generate a label.
num_sampled = 64  # Number of negative examples to sample.

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent. These 3 variables are used only for
# displaying model accuracy, they don't affect calculation.
valid_size = 16  # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
np.random.seed(1000)
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
print(valid_examples[:10])


[88  4 62 63 48 85 73 84 39 47]


In [39]:

graph = tf.Graph()

with graph.as_default():

    # Input data.
    with tf.name_scope('inputs'):
        train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
        train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
        valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
        

    # Ops and variables pinned to the CPU because of missing GPU implementation
    with tf.device('/cpu:0'):
        # Look up embeddings for inputs.
        with tf.name_scope('embeddings'):
            embeddings = tf.Variable(
                tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
            embed = tf.nn.embedding_lookup(embeddings, train_inputs)    
            print(embed.get_shape())

        # Construct the variables for the NCE loss
        with tf.name_scope('weights'):
            nce_weights = tf.Variable(
                tf.truncated_normal(
                    [vocabulary_size, embedding_size],
                    stddev=1.0 / math.sqrt(embedding_size)))
        with tf.name_scope('biases'):
            nce_biases = tf.Variable(tf.zeros([vocabulary_size]))
    
    with tf.device('/cpu:0'):
        ixa = tf.placeholder(tf.int32, shape=[1,])
        ixb = tf.placeholder(tf.int32, shape=[1,])
        ixc = tf.placeholder(tf.int32, shape=[1,])
        
    

    # Compute the average NCE loss for the batch.
    # tf.nce_loss automatically draws a new sample of the negative labels each
    # time we evaluate the loss.
    # Explanation of the meaning of NCE loss:
    #   http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/
    with tf.name_scope('loss'):
        loss = tf.reduce_mean(
            tf.nn.nce_loss(
                weights=nce_weights,
                biases=nce_biases,
                labels=train_labels,
                inputs=embed,
                num_sampled=num_sampled,
                num_classes=vocabulary_size))

    # Add the loss value as a scalar to summary.
    tf.summary.scalar('loss', loss)

    # Construct the SGD optimizer using a learning rate of 1.0.
    with tf.name_scope('optimizer'):
        optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

    # Compute the cosine similarity between minibatch examples and all embeddings.
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keepdims=True))
    normalized_embeddings = embeddings / norm
    valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings,
                                              valid_dataset)
    similarity = tf.matmul(
        valid_embeddings, normalized_embeddings, transpose_b=True)
    
    with tf.name_scope('analogy'):
        xa = tf.nn.embedding_lookup(normalized_embeddings, ixa)
        xb = tf.nn.embedding_lookup(normalized_embeddings, ixb)
        xc = tf.nn.embedding_lookup(normalized_embeddings, ixc)
        xtemp = xb - xa + xc
        hype = tf.matmul(xtemp, normalized_embeddings, transpose_b=True)
        hnorm = tf.sqrt(tf.reduce_sum(tf.square(hype), 1, keepdims=True))
        hype = hype / hnorm
        hypemax = tf.argmax(hype, 1)
        

    # Merge all summaries.
    merged = tf.summary.merge_all()

    # Add variable initializer.
    init = tf.global_variables_initializer()

    # Create a saver.
    saver = tf.train.Saver()


(128, 128)


In [65]:

# Step 5: Begin training.
num_steps = 200001

with tf.Session(graph=graph) as session:
    # Open a writer to write summaries.
    writer = tf.summary.FileWriter('./log', session.graph)

    # We must initialize all variables before we use them.
    init.run()
    print('Initialized')

    average_loss = 0
    for step in xrange(num_steps):
        batch_inputs, batch_labels = generate_batch(batch_size, num_skips,
                                                    skip_window)
        feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

        # Define metadata variable.
        run_metadata = tf.RunMetadata()

        # We perform one update step by evaluating the optimizer op (including it
        # in the list of returned values for session.run()
        # Also, evaluate the merged op to get all summaries from the returned "summary" variable.
        # Feed metadata variable to session for visualizing the graph in TensorBoard.
        _, summary, loss_val = session.run(
            [optimizer, merged, loss],
            feed_dict=feed_dict,
            run_metadata=run_metadata)
        average_loss += loss_val

        # Add returned summaries to writer in each step.
        writer.add_summary(summary, step)
        # Add metadata to visualize the graph for the last run.
        if step == (num_steps - 1):
            writer.add_run_metadata(run_metadata, 'step%d' % step)

        if step % 2000 == 0:
            if step > 0:
                average_loss /= 2000
            # The average loss is an estimate of the loss over the last 2000 batches.
            print('Average loss at step ', step, ': ', average_loss)
            average_loss = 0

        # Note that this is expensive (~20% slowdown if computed every 500 steps)
        if step % 10000 == 0:
            sim = similarity.eval()
            for i in xrange(valid_size):
                valid_word = reverse_dictionary[valid_examples[i]]
                top_k = 8  # number of nearest neighbors
                nearest = (-sim[i, :]).argsort()[1:top_k + 1]
                log_str = 'Nearest to %s:' % valid_word
                for k in xrange(top_k):
                    close_word = reverse_dictionary[nearest[k]]
                    log_str = '%s %s,' % (log_str, close_word)
#                 print(log_str)
    final_embeddings = normalized_embeddings.eval()
    
#     lines = open('semantic.txt', encoding='utf8').readlines()
# #     print(list(dictionary.keys())[10])
#     s = ''
#     correct = 0
#     for line in lines:
#         words = line[:-1].split(' ')
#         target = dictionary[words[3]]
#         ix = np.array([dictionary[words[0]],
#                         dictionary[words[1]],
#                         dictionary[words[2]]]).reshape((3, 1))
#         m = session.run([hypemax], feed_dict={ixa:ix[0], ixb:ix[1], ixc:ix[2]})
#         index = m[0][0]
#         rword = reverse_dictionary[index]
#         if index == target:
#             correct += 1
        
#         s += str(words) + ' ' + rword + '\n'
#     open('emb_result.txt', mode='w', encoding='utf-8').write(s)
#     print(correct/len(lines))
        

    # Write corresponding labels for the embeddings.
    with open('./log/metadata.tsv', 'w', encoding='utf8') as f:
        for i in xrange(vocabulary_size):
            f.write(reverse_dictionary[i] + '\n')

    # Save the model for checkpoints.
    saver.save(session, os.path.join('./log', 'model.ckpt'))

    # Create a configuration for visualizing embeddings with the labels in TensorBoard.
    config = projector.ProjectorConfig()
    embedding_conf = config.embeddings.add()
    embedding_conf.tensor_name = embeddings.name
    embedding_conf.metadata_path = os.path.join('./log', 'metadata.tsv')
    projector.visualize_embeddings(writer, config)

writer.close()


Initialized
Average loss at step  0 :  323.7934265136719
Average loss at step  2000 :  218.65706393814088
Average loss at step  4000 :  155.71951965904236
Average loss at step  6000 :  128.76652598190307
Average loss at step  8000 :  111.99491020202636
Average loss at step  10000 :  100.13319578456878
Average loss at step  12000 :  92.0726853237152
Average loss at step  14000 :  82.80389751720429
Average loss at step  16000 :  74.10824365329742
Average loss at step  18000 :  69.4550858464241
Average loss at step  20000 :  63.52254586172104
Average loss at step  22000 :  59.435683521270754
Average loss at step  24000 :  55.29181743812561
Average loss at step  26000 :  52.2200294072628
Average loss at step  28000 :  49.78369953489304
Average loss at step  30000 :  46.24521243238449
Average loss at step  32000 :  43.90232002854347
Average loss at step  34000 :  41.98488807272911
Average loss at step  36000 :  40.0310077958107
Average loss at step  38000 :  38.24536566257477
Average loss a

In [69]:
def distance(v, u):
    w = u - v
    return np.linalg.norm(w)

In [135]:
def analogy(embeds, a, b, c):
    a, b, c = dictionary[a], dictionary[b], dictionary[c]
    xa, xb, xc = embeds[a], embeds[b], embeds[c]
    xtemp = xc - xa + xb
#     print(distance(xb, (xc - xa)))
    print(distance(xc, xa))
    norm = np.linalg.norm(xtemp)
    cosine = embeds.dot(xtemp)/norm
    
#     diff = embeds.T - xtemp.reshape((128, 1))
    dots = cosine
#     dots = np.linalg.norm(diff)
    m = np.argmax(dots)
    return reverse_dictionary[m]


In [136]:

lines = open('syantax.txt', encoding='utf8').readlines()[:100]
s = ''
correct = 0
for line in lines:
    words = line[:-1].split(' ')
    pred = analogy(final_embeddings, words[0], words[1], words[2])
    s += str(words) + ' ' + pred + '\n'
    if words[-1] == pred:
        correct += 1
open('emb_result.txt', mode='w', encoding='utf-8').write(s)
print("Correct: {0} Accuracy: {1:.02}%".format(correct, (correct * 100 / len(lines))))

1.3935019
1.250398
1.3516086
1.338293
1.2572215
1.3213704
1.2874246
1.3020431
1.3542733
1.2940745
1.1940883
1.22345
1.3744129
1.2345681
1.310062
1.3122251
1.2478966
1.3322979
1.2962775
1.3923298
1.2394749
1.3959329
1.335829
1.3575401
1.3033873
1.3800788
1.2704324
1.2873193
1.3978508
1.4153293
1.352723
1.2299173
1.3501322
1.1769533
1.3980525
1.2975122
1.3035362
1.2604432
1.2216722
1.2663445
1.2957455
1.2633476
1.3192475
1.2945067
1.3252871
1.3110554
1.2327933
1.2559767
1.3497552
1.1998332
1.3640597
1.2431668
1.3361895
1.2761825
1.3935019
1.250398
1.3651628
1.2805184
1.2134044
1.3855375
1.1990708
1.2735643
1.3382615
1.2754579
1.2950308
1.2916105
1.332547
1.2914953
1.2281749
1.2467533
1.2892737
1.2996238
1.2905608
1.2964393
1.2397274
1.1745337
1.2283397
1.3110329
1.0965621
1.1707162
1.393526
1.2100364
1.2865999
1.2189028
1.2322885
1.31064
1.2407039
1.1916815
1.1939156
1.2745513
1.303247
1.2586777
1.1968764
1.2618601
1.3487206
1.3305887
1.2546877
1.2122451
1.301481
1.2417125
Correct: 0 Acc

In [ ]:

# pylint: disable=missing-docstring
# Function to draw visualization of distance between embeddings.
def plot_with_labels(low_dim_embs, labels, filename):
    assert low_dim_embs.shape[0] >= len(labels), 'More labels than embeddings'
    plt.rc('font', family='Abyssinica SIL')
    plt.figure(figsize=(18, 18))  # in inches
    for i, label in enumerate(labels):
        x, y = low_dim_embs[i, :]
        plt.scatter(x, y)
        plt.annotate(
            label,
            xy=(x, y),
            xytext=(5, 2),
            textcoords='offset points',
            ha='right',
            va='bottom')

    plt.savefig(filename)


try:
    # pylint: disable=g-import-not-at-top
    from sklearn.manifold import TSNE
    import matplotlib.pyplot as plt

    tsne = TSNE(
        perplexity=30, n_components=2, init='pca', n_iter=5000, method='exact')
    plot_only = 500
    low_dim_embs = tsne.fit_transform(final_embeddings[:plot_only, :])
    labels = [reverse_dictionary[i] for i in xrange(plot_only)]
    plot_with_labels(low_dim_embs, labels, 'tsne.png')

except ImportError as ex:
    print('Please install sklearn, matplotlib, and scipy to show embeddings.')
    print(ex)
